In [2]:
%%capture
!pip install "flair" -q
# !pip install "scispacy" -q
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz -q

In [ ]:
import pandas as pd
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings
from typing import List
# https://sagorbrur.github.io/2022/05/relation-extraction-using-flair/

In [ ]:
file_path = "/content/"
filename = "flair_train.txt"

In [ ]:
columns = {0:"text", 1:"ner"}

In [ ]:
corpus: Corpus = ColumnCorpus(file_path, columns,
                              train_file=filename)

2023-04-04 00:00:41,913 Reading data from /content
2023-04-04 00:00:41,915 Train: /content/flair_train.txt
2023-04-04 00:00:41,916 Dev: None
2023-04-04 00:00:41,917 Test: None


In [ ]:
print(len(corpus.train))

32818


In [ ]:
# print(corpus.train[0].to_tagged_string("ner"))

In [ ]:
tag_type = "ner"
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

<ipython-input-31-07532d47af76>:2: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


In [ ]:
embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

In [ ]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

2023-04-04 00:01:26,070 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2023-04-04 00:11:52,817 ----------------------------------------------------------------------------------------------------
2023-04-04 00:11:52,819 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-04-04 00:11:52,824 ----------------------------------------------------------------------------------------------------
2023-04-04 00:11:52,827 Corpus: "Corpus: 36464 train + 4052 dev + 26050 test sentences"
2023-04-04 00:11:52,829 ----------------------------------------------------------------------------------------------------
2023-04-04 00:11:52,83

100%|██████████| 815/815 [01:26<00:00,  9.41it/s]


2023-04-04 00:14:42,252 Evaluating as a multi-label problem: False
2023-04-04 00:14:43,552 0.0	0.0	0.0	0.0
2023-04-04 00:14:43,555 
Results:
- F-score (micro) 0.0
- F-score (macro) 0.0
- Accuracy 0.0

By class:
              precision    recall  f1-score   support

        Drug     0.0000    0.0000    0.0000   62717.0
    Strength     0.0000    0.0000    0.0000   43252.0
        Form     0.0000    0.0000    0.0000   41665.0
   Frequency     0.0000    0.0000    0.0000   36700.0
       Route     0.0000    0.0000    0.0000   30950.0
      Dosage     0.0000    0.0000    0.0000   23694.0
      Reason     0.0000    0.0000    0.0000   10250.0
    Duration     0.0000    0.0000    0.0000    2052.0
         ADE     0.0000    0.0000    0.0000    1463.0

   micro avg     0.0000    0.0000    0.0000  252743.0
   macro avg     0.0000    0.0000    0.0000  252743.0
weighted avg     0.0000    0.0000    0.0000  252743.0

2023-04-04 00:14:43,557 ------------------------------------------------------------

{'test_score': 0.0,
 'dev_score_history': [],
 'train_loss_history': [],
 'dev_loss_history': []}

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
final_model = "/content/resources/taggers/example-ner/final-model.pt"
model = SequenceTagger.load(final_model)
# create example sentence
sentence = Sentence('I have aspirin in my butt')
# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

In [4]:
from flair.datasets import RE_ENGLISH_CONLL04
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair. models import RelationExtractor
from flair.trainers import ModelTrainer

data_path = "/content/data"
# load corpus
corpus = RE_ENGLISH_CONLL04(data_path)
print(corpus)
# prepare label dictionary
label_dict = corpus.make_label_dictionary("relation")
print(label_dict)

# initialize embeddings
# if you have other embeddings, you can add below list also or replce
embedding_types = [
    # comment in these lines to use flair embeddings
    WordEmbeddings("glove"),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

extractor = RelationExtractor(embeddings=embeddings, 
                                entity_label_type="ner", 
                                label_dictionary=label_dict, 
                                label_type="relation")

trainer = ModelTrainer(extractor, corpus)

# start training
trainer.train("log_path",
            learning_rate=0.1,
            mini_batch_size=32,
            max_epochs=150)

2023-04-04 00:58:52,929 Reading data from /content/data/re_english_conll04
2023-04-04 00:58:52,932 Train: /content/data/re_english_conll04/conll04-train.conllu
2023-04-04 00:58:52,935 Dev: /content/data/re_english_conll04/conll04-dev.conllu
2023-04-04 00:58:52,937 Test: /content/data/re_english_conll04/conll04-test.conllu
2023-04-04 00:58:52,940 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-04-04 00:58:52,944 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-04-04 00:58:52,949 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 2 train + 2 dev + 2 test sentences
2023-04-04 00:58:52,952 Computing label dictionary. Progress:


2it [00:00, 1082.40it/s]

2023-04-04 00:58:52,964 Dictionary created for label 'relation' with 2 values: HOBBY (seen 2 times)
Dictionary with 2 tags: <unk>, HOBBY


2023-04-04 00:58:59,854 ----------------------------------------------------------------------------------------------------
2023-04-04 00:58:59,857 Model: "RelationExtractor(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (decoder): Linear(in_features=16784, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): CrossEntropyLoss()
  (weights): None
  (weight_tensor) None
)"
2023-04-04 00:58

/usr/local/lib/python3.9/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


...

2023-04-04 00:59:14,228 Evaluating as a multi-label problem: False
2023-04-04 00:59:14,248 DEV : loss 0.00014056757208891213 - f1-score (micro avg)  0.6667
2023-04-04 00:59:14,252 BAD EPOCHS (no improvement): 0


2023-04-04 00:59:15,021 ----------------------------------------------------------------------------------------------------
2023-04-04 00:59:15,025 Exiting from training early.
2023-04-04 00:59:15,027 Saving model ...
2023-04-04 00:59:17,268 Done.
2023-04-04 00:59:17,317 ----------------------------------------------------------------------------------------------------


100%|██████████| 1/1 [00:01<00:00,  1.25s/it]

2023-04-04 00:59:21,622 Evaluating as a multi-label problem: False
2023-04-04 00:59:21,648 0.5	1.0	0.6667	0.5
2023-04-04 00:59:21,650 
Results:
- F-score (micro) 0.6667
- F-score (macro) 0.5
- Accuracy 0.5

By class:
              precision    recall  f1-score   support

       HOBBY     1.0000    1.0000    1.0000         2
       <unk>     0.0000    0.0000    0.0000         0

   micro avg     0.5000    1.0000    0.6667         2
   macro avg     0.5000    0.5000    0.5000         2
weighted avg     1.0000    1.0000    1.0000         2

2023-04-04 00:59:21,653 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6666666666666666,
 'dev_score_history': [0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666